In [1]:
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from collections import Counter
from itertools import chain,combinations

In [1]:
# https://towardsdatascience.com/topic-modeling-with-bert-779f7db187e6

from collect_data import load_cache
from tokenizer import load_tokens

tokens = load_tokens('/data/blockchain-interoperability/blockchain-social-media/analysis/twitter/tokens/text')
data = load_cache('/data/blockchain-interoperability/blockchain-social-media/analysis/twitter/snapshots')

data['clean_text'] = [' '.join(t) for t in tokens]
data.sort_values('timestamp_ms')

loading tokens...:   0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/113 [00:00<?, ?it/s]

In [13]:
'asdfasdf'.upper()

'ASDFASDF'

In [8]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer


scores = SentimentIntensityAnalyzer().polarity_scores('this is one sucks so bad!')

In [12]:
max(scores.items(),key=lambda v: v[1] if v[0]!='compound' else -10 )

('neg', 0.645)

In [25]:
import torch

torch.cuda.device_count()

2

In [41]:
import transformers
pp = transformers.pipeline(
        'sentiment-analysis',
        model = 'cardiffnlp/twitter-roberta-base-sentiment',
        tokenizer='cardiffnlp/twitter-roberta-base-sentiment',
        truncation=True,
        max_length=512, 
        device=0,
    )

In [39]:
import pandas as pd
from tqdm.auto import tqdm

sample = pd.read_pickle('snapshots/00100000.pkl')


def transformer_sent_score(text,analyzer):
    # {'label': 'POS', 'score': 0.5677285194396973}
    scores = analyzer(text)[0]
    return scores['label'].upper(),scores['score']

for s in tqdm(sample.whole_text):
    transformer_sent_score(s,aa)
    break

  0%|          | 0/100000 [00:00<?, ?it/s]

In [1]:
from sentiment import load_sentiment


ss = load_sentiment('sentiment/vader')

loading sentiment..:   0%|          | 0/221 [00:00<?, ?it/s]

In [8]:
from embeddings import load_embeddings
from tqdm.auto import tqdm
from pathlib import Path
import torch
# ee = load_embeddings()

ee = [
    torch.load(f) 
    for f in tqdm(
        sorted(
            Path('embeddings/all-MiniLM-L6-v2/').glob('*.pkl')
        ),
        desc='loading embeddings..',
        leave=False
    )
]

loading embeddings..:   0%|          | 0/221 [00:00<?, ?it/s]

In [14]:
from dataset import TwitterDataset
dset = TwitterDataset('snapshots','tokens/text/','embeddings/all-MiniLM-L6-v2/','sentiment/vader/')

reading in snapshots..:   0%|          | 0/221 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [6]:
import pandas as pd
dd = pd.to_datetime(df.timestamp_ms,unit='ms')

In [10]:
import numpy as np


idxs = np.argsort(dd[:100])

In [12]:
np.array(dd[:100])[idxs]

array(['2022-11-11T12:03:53.882000000', '2022-11-11T12:03:53.946000000',
       '2022-11-11T12:03:53.965000000', '2022-11-11T12:03:54.075000000',
       '2022-11-11T12:03:54.086000000', '2022-11-11T12:03:54.183000000',
       '2022-11-11T12:03:54.198000000', '2022-11-11T12:03:54.265000000',
       '2022-11-11T12:03:54.288000000', '2022-11-11T12:03:54.336000000',
       '2022-11-11T12:03:54.375000000', '2022-11-11T12:03:54.396000000',
       '2022-11-11T12:03:54.430000000', '2022-11-11T12:03:54.461000000',
       '2022-11-11T12:03:54.468000000', '2022-11-11T12:03:54.547000000',
                                 'NaT', '2022-11-11T12:03:54.625000000',
       '2022-11-11T12:03:57.917000000', '2022-11-11T12:03:54.680000000',
       '2022-11-11T12:03:54.587000000', '2022-11-11T12:03:54.715000000',
       '2022-11-11T12:03:54.763000000', '2022-11-11T12:03:54.854000000',
       '2022-11-11T12:03:54.811000000', '2022-11-11T12:03:54.818000000',
       '2022-11-11T12:03:54.904000000', '2022-11-11

In [1]:
class something:
    def __init__(self):
        pass

    def __get__(self):
        return 1

In [2]:
something()[0]

TypeError: 'something' object does not support indexing

In [1]:
from dataset import load_dataset

dd = load_dataset('snapshots','tokens/text','embeddings/all-MiniLM-L6-v2/','sentiment/vader')

reading in snapshots..:   0%|          | 0/221 [00:00<?, ?it/s]

loading tokens...:   0%|          | 0/221 [00:00<?, ?it/s]

loading embeddings..:   0%|          | 0/221 [00:00<?, ?it/s]

loading sentiment..:   0%|          | 0/221 [00:00<?, ?it/s]

In [53]:
df = pd.DataFrame()



df[['sentiment','sentiment_score']] = ss

KeyboardInterrupt: 

In [58]:
df = pd.DataFrame()

# s = [(1,2)]
# df[['hi','bye']] = zip(*s)
s1,s2 = zip(*s)

df['hi'] = s1
df['bye'] = s2

In [59]:
df

,hi,bye
0,1,2


In [49]:
from dataset import load_dataset
data = load_dataset('snapshots','tokens/text/','sentiment/vader')

reading in snapshots..:   0%|          | 0/221 [00:00<?, ?it/s]

loading tokens...:   0%|          | 0/221 [00:00<?, ?it/s]

TypeError: expected str, bytes or os.PathLike object, not list

In [4]:
from torch import nn
import torch.nn.functional as F

class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
       
        #Encoder

        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2),
            nn.Conv2d(16, 4, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2),
        )
        # nn.MaxPool2d(2, 2)
       
        #Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(4, 16, 2, stride=2),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(16, 3, 2, stride=2),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))

In [12]:
class LinearAutoEncoder(nn.Module):
    def __init__(self):
        super(LinearAutoEncoder, self).__init__()
       
        #Encoder

        self.encoder = nn.Sequential(
            nn.Linear(384, 200),
            nn.ReLU(inplace=True),
            # nn.MaxPool2d(2,2),
            nn.Linear(200,50),
            nn.ReLU(inplace=True),
            # nn.MaxPool2d(2,2),
        )
        # nn.MaxPool2d(2, 2)
       
        #Decoder
        self.decoder = nn.Sequential(
            nn.Linear(50, 200),
            nn.ReLU(inplace=True),
            nn.Linear(200,384),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))
    

In [3]:
import pickle

pickle.load(open('sentiment/vader.pkl','rb'))

[]

In [20]:
lenc = lenc.cuda()
ebs = ebs.cuda()

from tqdm.auto import tqdm
from torch.optim import Adam

criterion = nn.MSELoss()
optimizer = Adam(lenc.parameters())

for epoch in tqdm(range(100)):
    optimizer.zero_grad()
    out = lenc(ebs)
    loss = criterion(out,ebs)
    loss.backward()
    optimizer.step()
    print('loss:',loss.item())

  0%|          | 0/100 [00:00<?, ?it/s]

loss: 0.002409222535789013
loss: 0.004125208128243685
loss: 0.0023909281007945538
loss: 0.002432329813018441
loss: 0.002463673474267125
loss: 0.0024721170775592327
loss: 0.0024827364832162857
loss: 0.0024854594375938177
loss: 0.00248575652949512
loss: 0.0024866077583283186
loss: 0.002486886689439416
loss: 0.0024863595608621836
loss: 0.0024846415035426617
loss: 0.002483841497451067
loss: 0.002484808675944805
loss: 0.002484197961166501
loss: 0.0024825020227581263
loss: 0.002481407020241022
loss: 0.002481244271621108
loss: 0.0024815055076032877
loss: 0.002481007482856512
loss: 0.0024796975776553154
loss: 0.002478713868185878
loss: 0.0024784798733890057
loss: 0.0024782337713986635
loss: 0.002477183472365141
loss: 0.0024760758969932795
loss: 0.002475673332810402
loss: 0.0024754612240940332
loss: 0.002474902430549264
loss: 0.0024742414243519306
loss: 0.0024734698235988617
loss: 0.002472821855917573
loss: 0.0024727489799261093
loss: 0.0024716167245060205
loss: 0.002471020445227623
loss: 0.002